In [1]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import KFold
from matplotlib import pyplot
from statistics import median
import pickle
import csv
import warnings
import datetime
import multiprocess
warnings.filterwarnings("ignore")

In [2]:
def get_median(data):
    data = sorted(data)
    size = len(data)
    if size % 2 == 0:  
        median = (data[size // 2] + data[size // 2 - 1]) / 2
        data[0] = median
    if size % 2 == 1:  
        median = data[(size - 1) // 2]
        data[0] = median
    return data[0]


In [3]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df


In [4]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t


In [5]:
def get_data(project_path, first_failures=True):
    columns = ['tr_build_id', 'git_num_all_built_commits', 'git_diff_src_churn', 'git_diff_test_churn', 'gh_diff_files_modified', 'tr_status']
    df = pd.read_csv(project_path, usecols = columns)
    df['tr_status'] = output_values(df['tr_status'])
    if first_failures:
        df = get_first_failures(df)
    print('Length of project is: {}'.format(len(df)))
    return df


In [6]:
def sbs(p_name):
    
    p = p_name
    result_file = 'version_results/' + p_name.split('.')[0] + '_sbs_predictions.csv'
    fileroot = '../../RQ2-Models/' + p_name.split('.')[0] + '_models/'
    
    pframe = pd.DataFrame()
    test_file = get_data('../data/full_data/' + p_name, first_failures=False)
    
    for i in range(1,11):
        ver = i
        
        if (p == 'vagrant.csv') | (p == 'puppet.csv'):
            if (ver == 1) | (ver == 2) :
                return
        if (p == 'fog.csv'):
            if (ver == 10):
                return
        if (p == 'graylog2-server.csv'):
            if (ver == 2) | (ver == 3) | (ver == 4) :
                return
        if (p == 'heroku.csv'):
            if (ver == 4):
                return
        if (p == 'orbeon-forms.csv'):
            if (ver == 8):
                return
        
        #retrieve the indexes
        filename = '../data/project_data_pickles/' + p_name + '_' + str(i) + '_indexes.pkl'
        with open(filename, 'rb') as save_file:
            train_build_ids = pickle.load(save_file)
            test_build_ids = pickle.load(save_file)
        
        
        #form the test df
        X_test = test_file [ test_file['tr_build_id'].isin(test_build_ids)] 
        y_test = X_test['tr_status'].tolist()
        
        X_test.drop('tr_status', inplace=True, axis=1)
        X_test.drop('tr_build_id', inplace=True, axis=1)
        
        
        #retrieve the model
        if len(y_test) > 0:        
            filename = fileroot + 'rq2_' + p_name.split('.')[0] + '_' + str(i) + '_best_model.pkl'
            model_file = open(filename, 'rb')
            forest = pickle.load(model_file)
        
        else:
            print('Not found for {}'.format(i))
            continue
        
        y_pred = forest.predict(X_test)
        
        verframe = pd.DataFrame()
        verframe['Build_Result'] = y_pred
        verframe['Actual_Result'] = y_test
        
        pframe = pframe.append(verframe)
    
    pframe.to_csv(result_file)
    return pframe

In [19]:
def get_results(pframe, project):
    
    if pframe is None:
        return []
    
    actual_results = pframe['Actual_Result'].tolist()
    pred_results = pframe['Build_Result'].tolist()
    
    first_failure = 0
    ci = []

    total_builds = len(actual_results)
    sbs_builds = 0
    
    #SBS Algorithm
    for i in range(len(actual_results)):

        #If first failure is already found, continue building until actual build pass is seen
        if first_failure == 1:
            ci.append(0)
            sbs_builds += 1

            if actual_results[i] == 1:
                #actual build pass is seen, switch to prediction
                first_failure = 0
            else:
                first_failure = 1
        else:
            #we're in prediction state, if predicted to skip, we skip
            if pred_results[i] == 1:
                ci.append(1)
            else:
                #if predicted to fail, we switch to determine state and set first_failure to True
                ci.append(0)
                sbs_builds += 1
                first_failure = 1-actual_results[i]


    total_builds = len(ci)
    actual_builds = ci.count(0)
    saved_builds = 100*ci.count(1)/total_builds
    reqd_builds = 100*ci.count(0)/total_builds
    
    if sbs_builds != actual_builds:
        print('PROBLEM!!')
    
    #computing delay
    delay_indexes = []
    built_indexes = []
    
    for i in range(len(ci)):
        if ci[i] == 0:
            built_indexes.append(i)
        else:
            if actual_results[i] == 0:
                delay_indexes.append(i)
    
    bp = 0
    mp = 0
    temp_delay = 0
    total_delay = 0
    
    delay_list = []
    while bp < len(built_indexes):
        while mp < len(delay_indexes) and delay_indexes[mp] < built_indexes[bp]:
            temp_delay = built_indexes[bp] - delay_indexes[mp]
            #print("Difference: {}, Built_index = {} , Missed_index = {}".format(temp_delay, built_indexes[bp], delay_indexes[mp]))
            total_delay += temp_delay
            delay_list.append(temp_delay)
            mp += 1
        bp += 1

    while mp < len(delay_indexes):
        temp_delay = total_builds - delay_indexes[mp]
        #print("Difference: {}, Built_index = {} , Missed_index = {}".format(temp_delay, total_builds, delay_indexes[mp]))
        total_delay += temp_delay
        delay_list.append(temp_delay)
        mp += 1
    
    #print(project, delay_list)
    for mp in delay_indexes:
        if mp in built_indexes:
            delay_list.append(0)
            print(p)
    #print(delay_list)
    


    delay = [total_delay]
#     print('required_builds is {}'.format(reqd_builds))
#     print('saved_builds is {}'.format(saved_builds))
#     print('length of ci is {}'.format(len(ci)))
#     print('delay is {}\n\n'.format(delay_list))
    
    lines = []
    
    for alg in algorithms:
        for b in batch_sizes:
            if alg == 'BATCH4':
                if b != 4:
                    continue
            
            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue
            
            lines.append([project, alg, b, saved_builds, reqd_builds, len(ci), delay_list, median(delay_list)])
    
    return lines   
    
    
    
    

In [20]:
projects = ['heroku.csv', 'vagrant.csv', 'opal.csv', 'cloudify.csv', 'cloud_controller_ng.csv', 'rubinius.csv', 'open-build-service.csv', 'gradle.csv', 'sonarqube.csv', 'loomio.csv', 'fog.csv', 'puppet.csv', 'concerto.csv', 'sufia.csv', 'geoserver.csv', 'orbeon-forms.csv', 'graylog2-server.csv', 'jruby.csv', 'metasploit-framework.csv']


In [21]:
algorithms = ['BATCH4', 'BATCHSTOP4', 'BATCHBISECT']
batch_sizes = [1, 2, 4, 8, 16]

In [22]:
lines = []
for p in projects:
    pframe = sbs(p)
    lines.extend(get_results(pframe, p))
    
#     for alg in algorithms:
#         for b in batch_sizes:
            
#             if alg == 'BATCH4':
#                 if b != 4:
#                     continue
#             if alg == 'BATCHSTOP4':
#                 if b < 4:
#                     continue
                    
#             l[1] = alg
#             l[2] = b
#             lines.append(l)
    

Length of project is: 2083
Length of project is: 4049
Length of project is: 2595
Length of project is: 4815
Length of project is: 2399
Length of project is: 4005
Length of project is: 3718
Length of project is: 3569
Length of project is: 3163
Length of project is: 2924
Length of project is: 2785
Length of project is: 2347
Length of project is: 2301
Length of project is: 2259
Length of project is: 2183
Length of project is: 2166
Length of project is: 2110
Length of project is: 8275
Length of project is: 7602


In [23]:
df = pd.DataFrame(lines, columns=['project', 'algorithm', 'batch_size', 'saved_builds', 'builds_reqd', 'testall_size', 'delay_list', 'median_delay'])


In [24]:
df

,project,algorithm,batch_size,saved_builds,builds_reqd,testall_size,delay_list,median_delay
0,opal.csv,BATCH4,4,98.611647,1.388353,2593,"[82, 81, 74, 52, 24, 23, 9, 158, 48, 47, 24, 2...",43.5
1,opal.csv,BATCHSTOP4,4,98.611647,1.388353,2593,"[82, 81, 74, 52, 24, 23, 9, 158, 48, 47, 24, 2...",43.5
2,opal.csv,BATCHSTOP4,8,98.611647,1.388353,2593,"[82, 81, 74, 52, 24, 23, 9, 158, 48, 47, 24, 2...",43.5
3,opal.csv,BATCHSTOP4,16,98.611647,1.388353,2593,"[82, 81, 74, 52, 24, 23, 9, 158, 48, 47, 24, 2...",43.5
4,opal.csv,BATCHBISECT,1,98.611647,1.388353,2593,"[82, 81, 74, 52, 24, 23, 9, 158, 48, 47, 24, 2...",43.5
...,...,...,...,...,...,...,...,...
112,metasploit-framework.csv,BATCHBISECT,1,99.342105,0.657895,7600,"[132, 49, 43, 19, 18, 215, 206, 201, 177, 161,...",127.5
113,metasploit-framework.csv,BATCHBISECT,2,99.342105,0.657895,7600,"[132, 49, 43, 19, 18, 215, 206, 201, 177, 161,...",127.5
114,metasploit-framework.csv,BATCHBISECT,4,99.342105,0.657895,7600,"[132, 49, 43, 19, 18, 215, 206, 201, 177, 161,...",127.5
115,metasploit-framework.csv,BATCHBISECT,8,99.342105,0.657895,7600,"[132, 49, 43, 19, 18, 215, 206, 201, 177, 161,...",127.5


In [25]:
df.to_csv('version_sbs_results.csv')